In [1]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine
import os
uri = os.environ.get('URI')


url = 'https://api.blockchain.info/charts/transactions-per-second?timespan=all&sampled=false&metadata=false&cors=true&format=json'

resp = requests.get(url)


In [2]:
data = pd.DataFrame(resp.json()["values"])

In [3]:
# from timestamp to date
data['x'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in data['x']]

In [4]:
data.head()

,x,y
0,2016-06-11 00:00:00,1.983333
1,2016-06-11 00:15:00,1.966667
2,2016-06-11 00:30:00,1.900000
3,2016-06-11 00:45:00,2.283333
4,2016-06-11 01:00:00,2.083333


In [5]:
data.x = pd.to_datetime(data.x)
data.head()

,x,y
0,2016-06-11 00:00:00,1.983333
1,2016-06-11 00:15:00,1.966667
2,2016-06-11 00:30:00,1.900000
3,2016-06-11 00:45:00,2.283333
4,2016-06-11 01:00:00,2.083333


In [6]:
from dotenv import load_dotenv

load_dotenv()
uri = os.getenv("URI")


In [7]:
engine = create_engine(uri)

In [8]:
# Currently we do not handle NON EXISTING TABLE error, so our database MUST have such tables
query = engine.execute('SELECT MAX(reality_date) FROM reality;')
last_reality_date = query.fetchall()[0][0]
query.close()

In [9]:
query = engine.execute("SELECT min(prediction_date), max(prediction_date) from predictions;")
prediction_date = query.fetchall()[0]
query.close()

In [10]:
first_prediction_date = prediction_date[0]
last_prediction_date = prediction_date[1]

if last_reality_date is None:
    date_extract = first_prediction_date # date_extract inidicates SINCE when we want to gather data
elif last_reality_date <= last_prediction_date:
    date_extract = last_reality_date
else:
    date_extract = last_reality_date

print(type(date_extract))
print(date_extract)

<class 'datetime.datetime'>
2024-03-08 00:00:00


In [11]:
# round to hours
data.x = data.x.dt.round("H")
data.tail(10)

/tmp/ipykernel_36423/3735830271.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data.x = data.x.dt.round("H")


,x,y
273901,2024-04-08 08:00:00,6.500000
273902,2024-04-08 08:00:00,5.733333
273903,2024-04-08 08:00:00,4.366667
273904,2024-04-08 08:00:00,8.383333
273905,2024-04-08 09:00:00,5.916667
273906,2024-04-08 09:00:00,8.816667
273907,2024-04-08 09:00:00,5.700000
273908,2024-04-08 10:00:00,6.966667
273909,2024-04-08 10:00:00,5.650000
273910,2024-04-08 10:00:00,5.616667


In [12]:
data_grouped = data.groupby("x").sum().reset_index()
data_grouped.tail()

,x,y
68566,2024-04-08 06:00:00,28.050000
68567,2024-04-08 07:00:00,15.666667
68568,2024-04-08 08:00:00,29.766667
68569,2024-04-08 09:00:00,20.433333
68570,2024-04-08 10:00:00,18.233333


In [13]:
print((data_grouped.tail(1).x) > datetime(2025,4,8))
type(datetime(2025,4,8))

68570    False
Name: x, dtype: bool


datetime.datetime

In [15]:
data_grouped = data_grouped.loc[data_grouped['x'] >= date_extract,:]

In [20]:
data_grouped.head()

,x,y
67816,2024-03-08 00:00:00,16.083333
67817,2024-03-08 01:00:00,8.700000
67818,2024-03-08 02:00:00,14.866667
67819,2024-03-08 03:00:00,14.883333
67820,2024-03-08 04:00:00,16.500000


In [19]:
upload_data = list(zip(data_grouped['x'], round(data_grouped['y'],4)))
upload_data[:3]

[(Timestamp('2024-03-08 00:00:00'), 16.0833),
 (Timestamp('2024-03-08 01:00:00'), 8.7),
 (Timestamp('2024-03-08 02:00:00'), 14.8667)]

In [24]:
upload_data[300:310]

[(Timestamp('2024-03-20 12:00:00'), 20.1333),
 (Timestamp('2024-03-20 13:00:00'), 11.3167),
 (Timestamp('2024-03-20 14:00:00'), 19.4),
 (Timestamp('2024-03-20 15:00:00'), 10.9833),
 (Timestamp('2024-03-20 16:00:00'), 19.75),
 (Timestamp('2024-03-20 17:00:00'), 11.35),
 (Timestamp('2024-03-20 18:00:00'), 15.6167),
 (Timestamp('2024-03-20 19:00:00'), 9.0667),
 (Timestamp('2024-03-20 20:00:00'), 18.6),
 (Timestamp('2024-03-20 21:00:00'), 11.1833)]